<a href="https://colab.research.google.com/github/andrewng88/ml_from_scratch/blob/master/4_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://www.geeksforgeeks.org/naive-bayes-classifiers/# 

In [1]:
import numpy as np

class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # init mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64) # 2 classes x 10 features
        self._var = np.zeros((n_classes, n_features), dtype=np.float64) # 2 x 10
        self._priors =  np.zeros(n_classes, dtype=np.float64) # 2 x 1
 
        # calculate mean, var, and prior for each class along the columns(axis=0)
        for idx, c in enumerate(self._classes):
            X_c = X[y==c]
            self._mean[idx, :] = X_c.mean(axis=0) #10 x 1
            self._var[idx, :] = X_c.var(axis=0) #10 x 1
            self._priors[idx] = X_c.shape[0] / float(n_samples) # overall probablity of classes 

    # predict individual row
    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class of each row
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx]) 

            # individual probability of each feature
            # we log it because of underflow/float issue
            posterior = np.sum(np.log(self._pdf(idx, x)))
            # instead of multiply we add since it's log-ged
            posterior = prior + posterior
            # we append both class 0 and class1
            posteriors.append(posterior)
            
        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]
    
    # https://www.youtube.com/watch?v=9CgOthUUdw4
    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(- (x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
    
        # predict all
    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

X, y = datasets.make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

nb = NaiveBayes()
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)

print("Naive Bayes classification accuracy", accuracy(y_test, predictions))

Naive Bayes classification accuracy 0.965
